<a href="https://colab.research.google.com/github/asigalov61/Meddleying-MAESTRO/blob/main/Meddleying_MAESTRO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Meddleying MAESTRO (ver 1.0)

***

## A full-featured Algorithmic Intelligence music generator with full multi-instrument MIDI support.

***

### Project Los Angeles

### Tegridy Code 2020

***

# Setup Environment, clone needed repos, and install all required dependencies

In [ ]:
#@title Install all dependencies (run only once per session)
!git clone https://github.com/asigalov61/arc-diagrams
!pip install pyknon
!pip install pretty_midi
!pip install pypianoroll
!pip install mir_eval
!apt install fluidsynth #Pip does not work for some reason. Only apt works
!pip install midi2audio
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 /content/font.sf2

!curl -L "https://github.com/asigalov61/MIDI-TXT-MIDI/raw/master/MIDI.py" > 'MIDI.py'

!mkdir Dataset

In [ ]:
#@title Import all modules

import time
import math

import tqdm.auto
import secrets

# For plotting
import mido
import librosa
import pretty_midi
import pypianoroll
from pypianoroll import Multitrack, Track
import matplotlib
import matplotlib.pyplot as plt
import mir_eval.display
import librosa.display
%matplotlib inline

from mido import MidiFile


from midi2audio import FluidSynth

from google.colab import output, drive

from IPython.display import display, Javascript, HTML, Audio, Image

%cd /content/arc-diagrams/
from arc_diagram import plot_arc_diagram
%cd /content/

ticks_per_note = 50
ctime = 0
cev_matrix = []

# Select and download a sample MIDI dataset

In [ ]:
#@title (BEST Choice / Multi-Intrumental) Original Tegridy MIDI DataSet (~450 MIDIs) 
%cd /content/Dataset/
!wget 'https://github.com/asigalov61/Tegridy-MIDI-Dataset/raw/master/Tegridy-MIDI-Dataset-Original.zip'
!unzip -j 'Tegridy-MIDI-Dataset-Original.zip'
!rm 'Tegridy-MIDI-Dataset-Original.zip'
%cd /content/

In [ ]:
#@title (OPTION 1) Download Google Magenta MAESTRO v.2.0.0 Piano MIDI Dataset (~1300 MIDIs)
%cd /content/Dataset/
!wget 'https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip'
!unzip -j maestro-v2.0.0-midi.zip
!rm maestro-v2.0.0-midi.zip
%cd /content/Dataset/

In [ ]:
#@title (OPTION 2) Download Tegridy special Piano/Violin MIDI dataset
%cd /content/Dataset/
!wget 'https://github.com/asigalov61/Tegridy-MIDI-Dataset/raw/master/Tegridy-MIDI-Dataset-CC-BY-NC-SA.zip'
!unzip '/content/Dataset/Tegridy-MIDI-Dataset-CC-BY-NC-SA.zip'
!rm '/content/Dataset/Tegridy-MIDI-Dataset-CC-BY-NC-SA.zip'
%cd /content/

# Process the Dataset

In [ ]:
#@title Convert MIDIs to Notes and MIDI Events Lists
debug = False #@param {type:"boolean"}


%cd /content/

# MIDI Dataset to txt dataset converter 
import MIDI
import os
import tqdm.auto


ev_matrix = []
not_matrix = []
durations_matrix = []
velocities_matrix = []


dataset_addr = "Dataset"
files = os.listdir(dataset_addr)
for file in tqdm.auto.tqdm(files):
    file_address = os.path.join(dataset_addr, file)


    score = []

    midi_file = open(file_address, 'rb')
    if debug: print('Processing File:', file_address)


    score1 = MIDI.midi2score(midi_file.read())
    midi_file.close()
    score2 = MIDI.score2opus(score1)
    score3 = MIDI.to_millisecs(score2)
    score = MIDI.opus2score(score3)

    itrack = 1
    while itrack < len(score):
        for event in score[itrack]:
          if event[0] == 'note':
            not_matrix.append(event[4])
            ev_matrix.append(event)

        itrack += 1
      


    if debug:
      print('File:', midi_file)

print('number of notes in the dataset:', len(notes_matrix))
print('Done!')
if debug:
  print('notes_matrix:', notes_matrix)
  print('notes total:', len(notes_matrix))
  print('notes events total:', len(ev_matrix))

# Custom MIDI / priming sequence option

In [ ]:
#@title Upload your custom MIDI here
full_path_to_MIDI_file = "/content/Adriano Celentano.Soli.kar" #@param {type:"string"}
MIDI_channels_selection = "all" #@param ["all"] {allow-input: true}
start_note_index = 200 #@param {type:"number"}
end_note_index = 500 #@param {type:"number"}
score = []
ctime = 0
midi_file = open(full_path_to_MIDI_file, 'rb')
if debug: print('Processing File:', file_address)

if MIDI_channels_selection == 'all':
  score1 = MIDI.midi2score(midi_file.read())
else:
  score0 = MIDI.midi2score(midi_file.read())
  score1 = MIDI.grep(score0, [int(MIDI_channels_selection)])
midi_file.close()
score2 = MIDI.score2opus(score1)
score3 = MIDI.to_millisecs(score2)
score = MIDI.opus2score(score3)
cnotes_matrix = []
cev_matrix = []
x = 0
itrack = 1
while itrack < len(score):
    for event in score[itrack]:
       if event[0] == 'note':
          if x >= start_note_index and x <= end_note_index: 
            cnotes_matrix.append(event[4])
          if x >= start_note_index and x <= end_note_index:    
            cev_matrix.append(['note', ctime, event[2], event[3], event[4], event[5]])
            ctime += ticks_per_note
          x += 1
    itrack += 1
  
if debug:
  print('File:', midi_file)

print('number of notes in the dataset:', len(cnotes_matrix))
print('Done!')

# Generate Music

In [ ]:
#@title Calculate all notes and make a MIDI output file 
start_note = 60 #@param {type:"number"}
notes_per_slice = 10 #@param {type:"slider", min:10, max:100, step:1}
number_of_slices = 150 #@param {type:"slider", min:2, max:1000, step:1}
ticks_per_note = 51 #@param {type:"slider", min:1, max:1000, step:10}
output_ticks = 201 #@param {type:"slider", min:1, max:1000, step:1}
absolute_or_relative_note_timings = False #@param {type:"boolean"}
set_all_MIDI_patches_to_piano = False #@param {type:"boolean"}
MIDI_channel_patch_00 = 0 #@param {type:"number"}
MIDI_channel_patch_01 = 25 #@param {type:"number"}
MIDI_channel_patch_02 = 33 #@param {type:"number"}
MIDI_channel_patch_03 = 41 #@param {type:"number"}
MIDI_channel_patch_04 = 43 #@param {type:"number"}
MIDI_channel_patch_05 = 47 #@param {type:"number"}
MIDI_channel_patch_06 = 57 #@param {type:"number"}
MIDI_channel_patch_07 = 72 #@param {type:"number"}
MIDI_channel_patch_08 = 74 #@param {type:"number"}
MIDI_channel_patch_09 = 0 #@param {type:"number"}
MIDI_channel_patch_10 = 0 #@param {type:"number"}
MIDI_channel_patch_11 = 0 #@param {type:"number"}
MIDI_channel_patch_12 = 0 #@param {type:"number"}
MIDI_channel_patch_13 = 0 #@param {type:"number"}
MIDI_channel_patch_14 = 0 #@param {type:"number"}
MIDI_channel_patch_15 = 0 #@param {type:"number"}


if set_all_MIDI_patches_to_piano:
  output = [output_ticks, [['track_name', 0, b'Composed by Meddleying MAESTRO']]]
else:
  output = [output_ticks, [['track_name', 0, b'Composed by Meddleying MAESTRO']], 
              [['patch_change', 0, 0, MIDI_channel_patch_00], 
               ['patch_change', 0, 1, MIDI_channel_patch_01],
               ['patch_change', 0, 2, MIDI_channel_patch_02],
               ['patch_change', 0, 3, MIDI_channel_patch_03],
               ['patch_change', 0, 4, MIDI_channel_patch_04],
               ['patch_change', 0, 5, MIDI_channel_patch_05],
               ['patch_change', 0, 6, MIDI_channel_patch_06],
               ['patch_change', 0, 7, MIDI_channel_patch_07],
               ['patch_change', 0, 8, MIDI_channel_patch_08],
               ['patch_change', 0, 9, MIDI_channel_patch_09],
               ['patch_change', 0, 10, MIDI_channel_patch_10],
               ['patch_change', 0, 11, MIDI_channel_patch_11],
               ['patch_change', 0, 12, MIDI_channel_patch_12],
               ['patch_change', 0, 13, MIDI_channel_patch_13],
               ['patch_change', 0, 14, MIDI_channel_patch_14],
               ['patch_change', 0, 14, MIDI_channel_patch_15],
               ]]
output_events_matrix = []
note_event = []
events_matrix = []

time = 0
if ctime > 0:
  time = ctime
else:
  time = 0

#priming_seq = priming_sequence.split(',')

if len(cev_matrix) != 0:
    events_matrix = cev_matrix + ev_matrix
    notes_matrix = cnotes_matrix + not_matrix
    start_note = cnotes_matrix[-1]
    index = notes_matrix.index(start_note, secrets.choice(range(len(notes_matrix))))
    #print('priming_sequence:', cev_matrix)
if start_note != 0 and start_note != '' and start_note < 128:
    events_matrix = ev_matrix
    notes_matrix = not_matrix
    index = notes_matrix.index(start_note, secrets.choice(range(len(notes_matrix))))
    print('priming_sequence: MIDI note #', [start_note])

#if start_note == 0:
#    start_note = secrets.choice(range(30, 90))
#    index = notes_matrix.index(start_note, secrets.choice(range(len(notes_matrix))))
#    print('priming_sequence: MIDI note #', [start_note])
#    print(index)

for i in tqdm.auto.tqdm(range(number_of_slices)):
  
  for k in range(notes_per_slice-4):
    if k > 4:
      event03 = events_matrix[index+k-4]
      event02 = events_matrix[index+k-3]
      event01 = events_matrix[index+k-2]
      event0 = events_matrix[index+k-1]

    event = events_matrix[index+k]
    event1 = events_matrix[index+k+1]
    event2 = events_matrix[index+k+2]
    event3 = events_matrix[index+k+3]
    if absolute_or_relative_note_timings: 
      time += abs(int(event1[2])-int(event[2]))
    else:
      time += ticks_per_note
    ovent = ['note', time, event[2], event[3], event[4], event[5]]
    output_events_matrix.append(ovent)
  #index = notes_matrix.index(event1[4], secrets.choice(range(len(notes_matrix))))
  for i in range(len(notes_matrix)-7):
    if notes_matrix[i] == event03[4]:
      if notes_matrix[i+1] == event02[4]:
        if notes_matrix[i+2] == event01[4]:
          if notes_matrix[i+3] == event0[4]:
            if notes_matrix[i+4] == event[4]:
              if notes_matrix[i+5] == event1[4]:
                if notes_matrix[i+6] == event2[4]:
                  if notes_matrix[i+7] == event3[4]:
                    index = i+7
#        else:
#          index = notes_matrix.index(event1[4], secrets.choice(range(len(notes_matrix))))
output += [cev_matrix + output_events_matrix]
#print(output)
#print('total notes:', x, 'check:', len(output[1]))

songt = MIDI.score2opus(output)
song = MIDI.to_millisecs(songt)
midi_data = MIDI.opus2midi(song)
if debug: print(song)


with open('output.mid', 'wb') as midi_file:
    midi_file.write(midi_data)
    midi_file.close()
print('Done!')

from google.colab import files
files.download('/content/output.mid')

MIDI.score2stats(song)

# Simple MIR stats

In [ ]:
#@title Analysis of the IO MIDI
MIDI_DIR = "/content/output.mid"
### https://github.com/brennan2602/FYP

#This file reads in the midi files in a directory, converts them to a string representation
#when in a string representation it then gathers some statistics about the structure of the song
import glob
import numpy as np

def get_piano_roll(midifile):
	midi_pretty_format = pretty_midi.PrettyMIDI(midifile)
	piano_midi = midi_pretty_format.instruments[0] # Get the piano channels
	piano_roll = piano_midi.get_piano_roll(fs=20)
	return piano_roll

#uses split encoding scheme (here only encoding the note values)
#works by looping through time increments of the piano roll array and writing the notes being played
#at a given time sample as a number on the corresponding line of a string # is written when no notes played for that
#sample
def encode(arr):
    timeinc=0
    outString=""
    for time in arr:
        notesinc = -1
        #print(time)
        if np.all(time==0):
            outString=outString+"#"
        for vel in arr[timeinc]:
            notesinc=notesinc+1
            if vel != 0:
                noteRep=str(notesinc) + " "
                #print(noteRep)
                outString=outString+noteRep
        outString=outString+"\n"
        timeinc = timeinc+1
    return outString


def getSilences(test):
    test=test[:-1] #removing last line in string (always blank)
    output=test.split("\n") #splitting into array
    res = len(output)
    #initialising counters
    maxcounter=0
    counter=0
    silenceCount=0

    for x in output:
        if x == "#": #when a "#" is seen nothing is being played that sample
            counter=counter+1 #this tracks a streak of silences
            silenceCount+=1 #this tracks total silences
        if x != "#":
            counter=0 #reseting streak
        if counter>maxcounter:
            maxcounter=counter #updating longest silence streak when appropriate
    return maxcounter,silenceCount


#by looking at the length of song and the amount of silences this returns % silence
def getPercentSilence(gen,silences):
    test = gen
    test = test[:-1]
    output = test.split("\n")
    res = len(output)
    percent=silences/res
    return percent


def getStatsNotes(test):
    test=test[:-1] #get rid of blank line at the end
    notes=[]
    output = test.split("\n") #split string on new lines

    #initial values updated while looping through
    maxPerSamp=0
    silenceSamp=0
    notesPlayed=0
    maxNotes=0
    maxVal=0
    minVal=127

    for x in output:
        samp=x.split(" ")
        samp=samp[:-1] #theres a blank result at the end of array from split this indexing removes it
        while "0" in samp:
            samp.remove("0") #sometimes 0 samples exist this removes them as they aren't notes played
        if len(samp)==0:
            silenceSamp+=1 #counting silences
        notesPlayed=notesPlayed+len(samp) #counting notes played
        if len(samp)>0:
            #getting max and min note values at this time step
            minimum=min(samp)
            maximum=max(samp)
            #updating max and min values note values for song if appropriate
            if int(minimum)<minVal:
                minVal=int(minimum)
            if int(maximum)>maxVal:
                maxVal=int(maximum)
        #updating maximum number of notes per sample if appropriate
        if len(samp)>maxNotes:
            maxNotes=len(samp)
    rangeNotes=maxVal-minVal #spread of notes
    avgNotes = notesPlayed / len(output) #average notes per sample
    adjNotes=notesPlayed /(len(output)-silenceSamp) #average notes per sample adjusted to remove silent samples
    return rangeNotes, maxVal, minVal,maxNotes,avgNotes,adjNotes


files=glob.glob(MIDI_DIR)#point towards directory with midi files (here same folder)
print(files)

for f in files:
    print(f)
    pr = get_piano_roll(f) #gets piano roll representation of the midi file
    arr = pr.T
    outString= encode(arr) #gets a string representation of the midi file
    maxsilences, silences = getSilences(outString) #by passing in the encoded string get longest silence and the total
                                                   #number of samples which are silent
    noteRange, maxVal, minVal, maxNotes, avgNotes, adjAvg =getStatsNotes(outString) # getting some stats by looping
                                                                                    # through encoded data
    percentSilence= getPercentSilence(outString,silences) # get % silence from silence / outString length

    #printing out to the user
    print("longest silence is ",maxsilences,"samples long")
    print("silence covers:",round(percentSilence,4),"%")
    print("notes span range:",noteRange)
    print("max note value:",maxVal)
    print("min note value:",minVal)
    print("average number of notes per sample:",round(avgNotes,4))
    print("average number of notes per sample (adjusted to remove silence samples):",round(adjAvg,4))
    print("max number of notes played in a sample:",maxNotes)
    print("\n")

#NOTE some minor discrepencies vs reading in from generated file directly
#However this does provide a uniform check to use for songs generated by both encoding schemes
#Can also be used to evaluate training file
#uses split encoding to get the text representation for ease of development

# Congrats! :) You did it :)

In [ ]:
#@title Make a nice Arc diagram of the output to show friends and family :)
multi_track_input = True #@param {type:"boolean"}
%cd /content/arc-diagrams/

midi_file = '/content/output.mid'
plot_title = "Meddleing MAESTRO Output Arc Diagram"

# midi_file = 'midis/fuer_elise.mid'
# plot_title = "Für Elise (Beethoven)"

def stringify_notes(midi_file, track_number):

    mid = MidiFile(midi_file)
    track_notes = {}
    for i, track in enumerate(mid.tracks):
        track_notes[i] = ''
        for msg in track:
            if( msg.type == 'note_on'):
                track_notes[i] += str(msg.note) +'n'
            if( msg.type == 'note_off'):
                track_notes[i] += str(msg.note) +'f'
    return track_notes[track_number]

if multi_track_input:
  try:
    plot_arc_diagram(stringify_notes(midi_file, 1), plot_title)
    if debug: 
      print('Debug mode')
    print('Track 1 Arc Diagram')
  except:
    plot_arc_diagram(stringify_notes(midi_file, 0), plot_title)
    if debug: 
      print('Debug mode')
    print('Track 0 Arc Diagram')

Image('output.png')

if multi_track_input:
  try:
   plot_arc_diagram(stringify_notes(midi_file, 2), plot_title)
   if debug: print('Debug mode')
   print('Track 2 Arc Digram')
   Image('output.png')
   plot_arc_diagram(stringify_notes(midi_file, 3), plot_title)
   if debug: print('Debug mode')
   print('Track 3 Arc Diagram')
   Image('output.png')
  except:
    print('Error in processing multiple tracks. Sorry.')
files.download('/content/arc-diagrams/output.png')

# MIDI Patch Numbers Reference Chart

***

## General MIDI Level 1 Instrument Families

### The General MIDI Level 1 instrument sounds are grouped by families. In each family are 8 specific instruments.

https://www.midi.org/specifications-old/item/gm-level-1-sound-set

***

## PC #	Family Name

1-8	Piano

9-16	Chromatic Percussion

17-24	Organ

25-32	Guitar

33-40	Bass

41-48	Strings

49-56	Ensemble

57-64	Brass

65-72	Reed

73-80	Pipe

81-88	Synth Lead

89-96	Synth Pad

97-104	Synth Effects

105-112	Ethnic

113-120	Percussive

121-128	Sound Effects

***

Note: While GM1 does not define the actual characteristics of any sounds, the names in parentheses after each of the synth leads, pads, and sound effects are, in particular, intended only as guides).

***

### PC #	Instrument Name
1.	Acoustic Grand Piano
2.	Bright Acoustic Piano
3.	Electric Grand Piano
4.	Honky-tonk Piano
5.	Electric Piano 1
6.	Electric Piano 2
7.	Harpsichord
8.	Clavi
9.	Celesta
10.	Glockenspiel
11.	Music Box
12.	Vibraphone
13.	Marimba
14.	Xylophone
15.	Tubular Bells
16.	Dulcimer
17.	Drawbar Organ
18.	Percussive Organ
19.	Rock Organ
20.	Church Organ
21.	Reed Organ
22.	Accordion
23.	Harmonica
24.	Tango Accordion
25.	Acoustic Guitar (nylon)
26.	Acoustic Guitar (steel)
27.	Electric Guitar (jazz)
28.	Electric Guitar (clean)
29.	Electric Guitar (muted)
30.	Overdriven Guitar
31.	Distortion Guitar
32.	Guitar harmonics
33.	Acoustic Bass
34.	Electric Bass (finger)
35.	Electric Bass (pick)
36.	Fretless Bass
37.	Slap Bass 1
38.	Slap Bass 2
39.	Synth Bass 1
40.	Synth Bass 2
41.	Violin
42.	Viola
43.	Cello
44.	Contrabass
45.	Tremolo Strings
46.	Pizzicato Strings
47.	Orchestral Harp
48.	Timpani
49.	String Ensemble 1
50.	String Ensemble 2
51.	SynthStrings 1
52.	SynthStrings 2
53.	Choir Aahs
54.	Voice Oohs
55.	Synth Voice
56.	Orchestra Hit
57.	Trumpet
58.	Trombone
59.	Tuba
60.	Muted Trumpet
61.	French Horn
62.	Brass Section
63.	SynthBrass 1
64.	SynthBrass 2
65.	Soprano Sax
66.	Alto Sax
67.	Tenor Sax
68.	Baritone Sax
69.	Oboe
70.	English Horn
71.	Bassoon
72.	Clarinet
73.	Piccolo
74.	Flute
75.	Recorder
76.	Pan Flute
77.	Blown Bottle
78.	Shakuhachi
79.	Whistle
80.	Ocarina
81.	Lead 1 (square)
82.	Lead 2 (sawtooth)
83.	Lead 3 (calliope)
84.	Lead 4 (chiff)
85.	Lead 5 (charang)
86.	Lead 6 (voice)
87.	Lead 7 (fifths)
88.	Lead 8 (bass + lead)
89.	Pad 1 (new age)
90.	Pad 2 (warm)
91.	Pad 3 (polysynth)
92.	Pad 4 (choir)
93.	Pad 5 (bowed)
94.	Pad 6 (metallic)
95.	Pad 7 (halo)
96.	Pad 8 (sweep)
97.	FX 1 (rain)
98.	FX 2 (soundtrack)
99.	FX 3 (crystal)
100.	FX 4 (atmosphere)
101.	FX 5 (brightness)
102.	FX 6 (goblins)
103.	FX 7 (echoes)
104.	FX 8 (sci-fi)
105.	Sitar
106.	Banjo
107.	Shamisen
108.	Koto
109.	Kalimba
110.	Bag pipe
111.	Fiddle
112.	Shanai
113.	Tinkle Bell
114.	Agogo
115.	Steel Drums
116.	Woodblock
117.	Taiko Drum
118.	Melodic Tom
119.	Synth Drum
120.	Reverse Cymbal
121.	Guitar Fret Noise
122.	Breath Noise
123.	Seashore
124.	Bird Tweet
125.	Telephone Ring
126.	Helicopter
127.	Applause
128.	Gunshot


